In [1]:
import numpy as np 

In [2]:
import scipy.linalg as la
def true_Singular(original_S, dims):
    true_S=np.zeros(dims)
    true_S[:original_S.size, :original_S.size]=np.diag(original_S)
    return true_S

# $|\Psi\rangle$-state construction

En esta sección, estuve buscando una forma alternativa y capaz más eficiente (al menos para $N$-grande) de la construcción del 
tensor, no lo logré (aún) pero otro día le pego una vuelta. 

In [8]:
def extract_digits(x, d, N):
    alphas = []
    for _ in range(N):
        alpha_k = x % d    # Extract the remainder (alpha_k)
        alphas.insert(0,alpha_k)
        x = x // d         # Update x by removing the current digit
    return alphas

# Example usage
x = 6   # Number to decompose
d = 2      # Base
N = 5       # Number of coefficients to extract

alphas = extract_digits(x, d, N)
print(alphas)  # This will print the list of alpha_k coefficients

def extract_digits_2(x, d, N): 
    alphas = []
    for _ in range(N):
        alpha_k = x % d**2    # Extract the remainder (alpha_k)
        #alphas.append(alpha_k)
        alphas.insert(0,alpha_k)
        x = x // d**2         # Update x by removing the current digit
    alphas2 = []
    for i in range(N):        #writes alpha_k=(i_j,j_k) 
        i_k = alphas[i]//d %d
        alphas2.append(i_k)
        j_k = alphas[i]%d
        alphas2.append(j_k)
    return alphas2

def tensor_function(x,d,N):
    alphas=extract_digits_2(x,d,N)
    num = 1
    for i in range(0,N-1):
        if alphas[2*i+2]!=alphas[2*i+1] or alphas[0]!=alphas[2*N-1]: 
           num=num*0
    return num
    
x = 6   # Number to decompose
d = 2      # Base
N = 3       # Number of coefficients to extract

alphas = extract_digits(x, d**2, N)
alphas2 = extract_digits_2(x, d, N)

[0, 0, 1, 1, 0]


In [9]:
def swap_tensor_identity_flat(state, indices):
    # Ensure the state is a NumPy array
    state = np.array(state)
    n = len(state)
    i, j = indices
    if i >= n // 2 or j >= n // 2 or i < 0 or j < 0:
        raise ValueError("Indices must be within the range of pairs.")

    # Create the swapped state
    swapped_state = np.copy(state)
    
    # Perform the SWAP * Identity operation on pairs
    swapped_state[2*i], swapped_state[2*j] = state[2*j], state[2*i]
    swapped_state[2*i+1], swapped_state[2*j+1] = state[2*j+1], state[2*i+1]

    return swapped_state
    
state = np.array([1., "a", 2., "b", 3., "c", 4., "d"])  # |ij>_0 |ab>_1 format
swapped_state = swap_tensor_identity_flat(state, (0, 1))

print("Original state:", state)
print("SWAP 0,1 result:", swapped_state)

Original state: ['1.0' 'a' '2.0' 'b' '3.0' 'c' '4.0' 'd']
SWAP 0,1 result: ['2.0' 'b' '1.0' 'a' '3.0' 'c' '4.0' 'd']


In [10]:
import itertools
N=2 ## number of temporal slices
d=2 ## internal, physical, dimension 

total_states = d ** (2 * N)
# This function generates the correct weights for the Phi+ superstate 
def is_valid_state(state):
    binary_state = bin(state)[2:].zfill(2 * N)  # Binary representation of the state
    return all(binary_state[i:i+2] == '00' or binary_state[i:i+2] == '11' for i in range(0, len(binary_state), 2))

weights = np.array([1 if is_valid_state(i) else 0 for i in range(total_states)])
weights

array([1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1])

swap_tensor_identity_flat(weights, )

### MPS Decomposition 

In [11]:
d=2 #Size of the original Hilbert space (d=2 qubit; d=3 qutrit)
N=3 #Number of time slices
tensor = np.array([tensor_function(x,d,N) for x in np.arange(0,d**(2*N))])
print(tensor.shape)

(64,)


In [12]:
tensor.reshape(d**2, -1).shape

(4, 16)

In [15]:
def mps_decomposition(state_weights, N, d, bond_dim=None):
    reshaped_matrix = state_weights.reshape(d**2, -1)
    mps_matrices = []  # To store U matrices
    svd_vals = []      # To store singular values
    all_matrices = []
    
    current_matrix = reshaped_matrix
    for n in range(0, N-1):
        
        # Perform SVD with full_matrices=True
        U, S, Vh = np.linalg.svd(current_matrix, full_matrices=True)
        
        # Store U and S values
        mps_matrices.append(U)
        svd_vals.append(S)

        # Update current_matrix for the next iteration
        current_matrix = (true_Singular(S, (U.shape[1], Vh.shape[0])) @ Vh)
        
        # Reshape current_matrix correctly (rows reduce after SVD)
        current_matrix = current_matrix.reshape(Vh.shape[0], U.shape[0])
        all_matrices.append(current_matrix)

        # Check if there's a size issue for the next iteration
        if current_matrix.shape[0] == 0:
            print("Terminated early due to zero rows.")
            break

    # Append the final matrix product of S and Vh to mps_matrices
    current_matrix = current_matrix.reshape(U.shape[0], Vh.shape[0])
    all_matrices.append(current_matrix) ## Just to make sure, we extract all matrices involved.
    mps_matrices.append(current_matrix) ## append the last matrix as an MPS matrix 

    return svd_vals, mps_matrices, all_matrices

In [21]:
## solución a mano: 
U1, S1, Vh1=np.linalg.svd(tensor.reshape(4, 16))
U2, S2, Vh2=np.linalg.svd((true_Singular(S1, (4,16)) @ Vh1).reshape(16,4))
U3 = true_Singular(S2, (16,4))@Vh2 
#U2, S2, Vh2=np.linalg.svd((true_Singular(S1, (4, 16))@Vh1).reshape(16,4))
#U3_F=true_Singular(S2, (4, 16))@Vh2

In [22]:
svd_vals, mps_matrices, all_matrices =mps_decomposition(tensor, 3, 2, np.inf)

In [54]:
bigTensor= np.array([[
    [sum(mps_matrices[0][k0, a1] * mps_matrices[1][k1*4+a1, a2] * mps_matrices[2][k2, a2]  ## 99% seguro que esto está bien
        for a1 in range(d**2)
        for a2 in range(d**2))
      for k2 in range(d**2)]
     for k1 in range(d**2)]
for k0 in range(d**2)])

In [56]:
bigTensor

array([[[ 0.00000000e+00, -7.07106781e-01,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [-2.23711432e-17,  0.00000000e+00,  0.00000000e+00,
         -7.07106781e-01],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00]],

       [[-7.07106781e-01,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00, -7.07106781e-01,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00]],

       [[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00, -7.07106781e-01,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  5.61334798e-17,  0.00000000e+00,
          0.00000000e+00],
        [ 1.01465364e-17,  0.00000000e+

In [ ]:
result = np.array([[
    [sum(A1[ke, a1] * A2[k1 * 4 + a1, a2] * A3[k2, a2] 
         for a1 in range(4) 
         for a2 in range(4))
     for k2 in range(4)]
    for k1 in range(4)]
for ke in range(4)])

In [37]:
mps_matrices[2]

array([[ 0.        ,  0.        , -1.41421356,  0.        ],
       [ 0.        ,  0.        ,  0.        , -1.41421356],
       [-1.41421356,  0.        ,  0.        ,  0.        ],
       [ 0.        , -1.41421356,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ]])

In [403]:
tensor.shape

(64,)

In [332]:
U3_F

array([[-1.41421356,  0.        ,  0.        ,  0.        ,  0.        ,
        -1.41421356,  0.        ,  0.        ,  0.        ,  0.        ,
        -1.41421356,  0.        ,  0.        ,  0.        ,  0.        ,
        -1.41421356],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ]])

In [227]:
S2

array([2.82842712, 0.        , 0.        , 0.        ])

In [79]:
print(U2)

[[1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]


In [50]:
svd_vals, mps_decomp=mps_decomposition(tensor, 3, 2, 4)

In [51]:
svd_vals

[array([1.41421356, 1.41421356, 1.41421356, 1.41421356]),
 array([1.41421356, 1.41421356, 1.41421356, 1.41421356]),
 array([1.41421356, 1.41421356, 1.41421356, 1.41421356])]

In [29]:
mps_decomp[0]

array([[-0.5,  0. ],
       [ 0. ,  0. ],
       [ 0. ,  0. ],
       [-0.5,  0. ],
       [ 0. ,  0. ],
       [ 0. ,  0. ],
       [ 0. ,  0. ],
       [ 0. ,  0. ],
       [ 0. ,  0. ],
       [ 0. ,  0. ],
       [ 0. ,  0. ],
       [ 0. ,  0. ],
       [-0.5,  0. ],
       [ 0. ,  0. ],
       [ 0. ,  0. ],
       [-0.5,  0. ],
       [ 0. , -0.5],
       [ 0. ,  0. ],
       [ 0. ,  0. ],
       [ 0. , -0.5],
       [ 0. ,  0. ],
       [ 0. ,  0. ],
       [ 0. ,  0. ],
       [ 0. ,  0. ],
       [ 0. ,  0. ],
       [ 0. ,  0. ],
       [ 0. ,  0. ],
       [ 0. ,  0. ],
       [ 0. , -0.5],
       [ 0. ,  0. ],
       [ 0. ,  0. ],
       [ 0. , -0.5]])

In [55]:
N=4 # number of copies 
d=2 # internal dimensionality of the system

Phi_plus=np.array([ [np.random.rand() for i in range(-1,d-1)] for j in range(N)])
local_state=Phi_plus
local_state=swap_tensor_identity(local_state, (3,0))
local_state=swap_tensor_identity(local_state, (2,3))
local_state=swap_tensor_identity(local_state, (1,2))

In [58]:
Phi_plus

array([[0.5421803 , 0.10684483],
       [0.76674341, 0.40250495],
       [0.98470778, 0.8753654 ],
       [0.04026931, 0.98993984]])

In [59]:
local_state

array([[0.04026931, 0.10684483],
       [0.5421803 , 0.40250495],
       [0.76674341, 0.8753654 ],
       [0.98470778, 0.98993984]])

In [269]:
import numpy as np

def mps_decomposition(state_weights, N, d, bond_dim):
    # Reshape the state weights into a 2D matrix of shape (d^N, d)
    reshaped_matrix = state_weights.reshape(-1, d)
    mps_matrices = []  # To store U matrices
    svd_vals = []      # To store singular values
    
    current_matrix = reshaped_matrix
    for n in range(N):
        # Perform SVD
        U, S, Vh = np.linalg.svd(current_matrix, full_matrices=False)

        # Apply bond dimension cutoff
        if bond_dim < len(S):
            U = U[:, :bond_dim]   # Keep only the first 'bond_dim' columns of U
            S = S[:bond_dim]      # Keep only the first 'bond_dim' singular values
            Vh = Vh[:bond_dim, :] # Keep only the first 'bond_dim' rows of Vh

        # Append U and singular values to their respective lists
        mps_matrices.append(U)
        svd_vals.append(S)

        # Reconstruct the current_matrix for the next iteration
        current_matrix = np.dot(np.diag(S), Vh)

        # Check if current_matrix has valid dimensions for the next SVD
        if current_matrix.shape[0] == 0:  # No remaining rows
            break
    
    return svd_vals, mps_matrices

# Example usage
weights_full = np.zeros(256)
weights_full[0] = 1  # Example input state |0000>
weights_full[3] = 1  # Example input state |0011>
weights_full[12] = 1  # Example input state |1100>
weights_full[15] = 1  # Example input state |1111>

N = 2  # Number of copies
d = 2  # Physical dimension
bond_dim = 5  # Set bond dimension cutoff

svd_values, mps_matrices = mps_decomposition(weights_full, N, d, bond_dim)

# Print the SVD values and MPS matrices
print("SVD Values:")
for i, sv in enumerate(svd_values):
    print(f"SVD {i}:", sv)

print("\nMPS Matrices:")
for i, matrix in enumerate(mps_matrices):
    print(f"MPS Matrix {i}:\n", matrix)

SVD Values:
SVD 0: [1.41421356 1.41421356]
SVD 1: [1.41421356 1.41421356]

MPS Matrices:
MPS Matrix 0:
 [[-0.70710678  0.        ]
 [ 0.         -0.70710678]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [-0.70710678  0.        ]
 [ 0.         -0.70710678]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.

In [121]:
Phi_plus

array([[0.42758846, 0.49951572],
       [0.74313709, 0.52018825],
       [0.22730234, 0.26888561],
       [0.07200348, 0.07470163]])

In [122]:
local_state

array([[0.42758846, 0.49951572],
       [0.22730234, 0.52018825],
       [0.07200348, 0.26888561],
       [0.74313709, 0.07470163]])

In [93]:
Phi_plus=swap_tensor_identity(Phi_plus, (1,2))
Phi_plus

array([[0.67769074, 0.10646232],
       [0.20935205, 0.12720757],
       [0.70763754, 0.05422026],
       [0.59056801, 0.41011425]])